### 통계치 확인

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import pandas as pd
df=pd.read_csv('민원.csv', encoding='cp949')
df=df.drop_duplicates().dropna()
df=df.drop(['처리상태','가상번호'], axis=1)
df['등록일자']=df['등록일자'].str[6:]
df['등록일자']=pd.to_datetime(df['등록일자'])
df['날짜_월']=df['등록일자'].dt.month
df['날짜_일']=df['등록일자'].dt.day
df=df.drop('등록일자', axis=1)

분류_갯수=df.groupby(['분류'])['제목'].count()
분류_갯수.to_csv('분류_갯수.csv', encoding='cp949', index=False)

df['분기']=pd.cut(df['날짜_월'],bins=[0,3,6,9,13], labels=[1,2,3,4])
분류_분기_갯수=df.groupby(['분류','분기'])['제목'].count()
분류_분기_갯수.to_csv('분류_분기_갯수.csv', encoding='cp949', index=False)

In [3]:
분류_unique=df['분류'].unique()
for i in 분류_unique:
    print(i)
    df_분류=df[df['분류']==i]
    df_분류.to_csv('분류/분류_'+i+'.csv', encoding='cp949', index=False)

건설ㆍ교통
기타
보건ㆍ복지
도시ㆍ건축
미담사례
환경ㆍ청소
공원ㆍ체육
문화ㆍ경제


### topic_model_전체

In [4]:
data=df[['제목']]
data=data.drop_duplicates().dropna()
data_=data['제목'].values.tolist()
import re
regex=[]

for i in range(len(data_)):
    text=re.sub('[^0-9ㄱ-힣]', ' ', str(data_[i]))
    text=re.sub(' +', ' ', text)
    regex.append(text)
regex
#명사추출
#!pip install konlpy
from konlpy.tag import Okt
okt=Okt()

noun=[]
for j in range(len(regex)):
    noun.append(okt.nouns(regex[j]))

    
# !pip install gensim
import gensim

id2word = gensim.corpora.Dictionary(noun)
texts=noun.copy()

for index,word in enumerate(texts):
    for i, w in enumerate(word):
        if len(w)<2:
            texts[index].pop(i)
#doc2bos-> 단어를 숫자로 변환
corpus=[id2word.doc2bow(text) for text in texts]

# topic 개수 몇개로 할지
number=5

model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                        id2word=id2word, 
                                        num_topics=number, 
                                        random_state=25)
model.show_topics(formatted=False)
#시각화 2.1.2버전이 잘 작동함
#!pip install pyLDAvis==2.1.2
import pyLDAvis.gensim

vis=pyLDAvis.gensim.prepare(model,corpus, id2word)
pyLDAvis.save_html(vis,'LDA/LDA_전체.html')

In [5]:
분류_unique

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['건설ㆍ교통', '기타', '보건ㆍ복지', '도시ㆍ건축', '미담사례', '환경ㆍ청소', '공원ㆍ체육', '문화ㆍ경제'],
      dtype=object)

In [6]:
# 분류별 파일 만들기
for m in 분류_unique:
    df=pd.read_csv('분류/분류_'+m+'.csv', encoding='cp949')
    data=df[['제목']]
    data=data.drop_duplicates().dropna()
    data_=data['제목'].values.tolist()
    
    import re
    regex=[]

    for i in range(len(data_)):
        text=re.sub('[^0-9ㄱ-힣]', ' ', str(data_[i]))
        text=re.sub(' +', ' ', text)
        regex.append(text)
    regex
    #명사추출
    #!pip install konlpy
    from konlpy.tag import Okt
    okt=Okt()

    noun=[]
    for j in range(len(regex)):
        noun.append(okt.nouns(regex[j]))

    #!pip install gensim
    import gensim

    id2word = gensim.corpora.Dictionary(noun)
    texts=noun.copy()

    for index,word in enumerate(texts):
        for i, w in enumerate(word):
            if len(w)<2:
                texts[index].pop(i)
    #doc2bos-> 단어를 숫자로 변환
    corpus=[id2word.doc2bow(text) for text in texts]

    # topic 개수 몇개로 할지
    number=5

    model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word, 
                                            num_topics=number, 
                                            random_state=25)
    model.show_topics(formatted=False)
    #시각화 2.1.2버전이 잘 작동함
    #!pip install pyLDAvis==2.1.2
    import pyLDAvis.gensim

    vis=pyLDAvis.gensim.prepare(model,corpus, id2word)
    pyLDAvis.save_html(vis,'LDA/LDA_'+m+'.html')

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
